In [1]:
import pandas as pd
import numpy as np
import os, sys
import time

In [2]:
'''
# Table with information about the dataset:
# https://www.nature.com/articles/s41597-019-0219-y/tables/2?draft=collection

Download the dataset and place it in a folder that is reachable from this notebook.

Set DATA_PATH to the filename of the CSV file.

You can use os.listdir('./') to see which files are in the same directory as your notebook
'''

print(os.listdir('./'))
DATA_PATH = './190516_m2b_marine_biota.csv'

['yellow.txt', 'blue.txt', 'dataset_100_rows.csv', 'csv_parser.ipynb', 'readme.md', 'red.txt', '.ipynb_checkpoints', '190516_m2b_marine_biota.csv', 'dataset.csv', '190516_m2b_marine_biota.json']


In [5]:
df = pd.read_csv(DATA_PATH, delimiter=";")

relevant_columns = [
    'samp_date', # renamed to 'years' later
    'country',
    'mea_ug_kg_orig', # mean mercury concentration
    'troph_lev',
    'mar_habit',
    'location',
    'lenght_cm',
    'weight_g',
    'tissue_cod',
    'spec_nam_s',
    'hg_species'
    # 'age_y' # so many blank rows that we leave this one out
]

df = df[relevant_columns].copy()
df.dropna() # remove missing values

# parse dates to integer years and rename column to 'years'
df['samp_date'] = df['samp_date'].apply(lambda s_time: time.strptime(s_time, '%Y-%m-%d').tm_year)
df = df.rename(columns={'samp_date': 'year'})

# remove all rows whose numerical col value == -9999.0, which is the none type format in the dataset
print(f"Shape before trim: {df.shape}")
for col in df.columns:
    df = df[df[col] != -9999.0]
print(f"Shape after trim: {df.shape}")

# encode string valued columns to integer ids
cols_to_encode = [
    'country',
    'mar_habit',
    'location',
    'spec_nam_s',
    'tissue_cod',
    'hg_species'
]

# TO DO: make a remainder dataframe for the keys so that it's interpretable what the data mean
for col in cols_to_encode:
    df[col], key = df[col].factorize() # key corresponds to e.g. country codes (IT, ES, MR etc.)

df.sort_values(by=['year'], inplace=True)

# restart row count
df = df.reset_index()
df = df.drop(columns=['index'])

# export to CSV
print(f"Exporting dataset with {df.shape[1]} columns and {df.shape[0]} rows")
# df.to_csv("dataset.csv", index_label='id')
df[:100].to_json("dataset.json")

Shape before trim: (24465, 11)
Shape after trim: (9863, 11)
Exporting dataset with 11 columns and 9863 rows


In [82]:
# df.loc['year']
'''
print(df['year'].unique())
mean_hg = df[df['year'] == 1971]['mea_ug_kg_orig'].mean()
print(mean_hg)
df
'''
# df.loc[:,'mea_ug_kg_orig'] = mean_hg

years = []
for year in list(df['year'].unique()):
    if len(df[df['year'] == year]) > 9:
        years.append(year)

print(df[df['year'] == 1971]['country'].unique())
                
'''
# a set of subsequent years that all have more than 9 entries
years = [1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 
         1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 
         1990, 1991, 1992, 1993, 1995, 1996, 1997, 1998, 1999, 
         2000, 2001, 2002, 2003, 2004, 2005, 2006, 2008, 2010, 2011, 2012]        
'''

[3 6 5 1]


'\n# a set of subsequent years that all have more than 9 entries\nyears = [1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, \n         1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, \n         1990, 1991, 1992, 1993, 1995, 1996, 1997, 1998, 1999, \n         2000, 2001, 2002, 2003, 2004, 2005, 2006, 2008, 2010, 2011, 2012]        \n'

In [ ]:
'''
This block splits dataset into blue, yellow and red of the following format:

[blue]: index, year col1 col2 col3
[red]: index, year col4 col5 col6
[yellow]: index, year col7 col8 col9

'''


# only use 20 counties
df = df[df['country'] < 20]

cols = df.columns.to_list()[2:] # don't use year and country
idx = len(cols) // 3 # number of cols must be a multiple of 3 as of now

index = [str(i) + "," for i in list(np.arange(1, df.shape[0] + 1))]
df['index'] = index
index_col = df.pop('index')
df.insert(0, 'index', index_col)

colors = ['blue', 'red', 'yellow']
rgb_cols = [cols[idx*i:idx*(i+1)] for i, c in enumerate(colors)]
rgb_df = [df[['index', 'year', 'country'] + rgb_cols[i]] for i, c in enumerate(rgb_cols)]

for i, frame in enumerate(rgb_df):
    print(f"Exporting dataset {colors[i]}.txt with {frame.shape[1]} columns and {df.shape[0]} rows")
    frame.to_csv(f"{colors[i]}.txt", 
        sep=" ", 
        line_terminator= ";" + os.linesep, # os.linesep is the native newline
        header=False,
        index=False
    )